In [1]:
import pickle
import numpy as np


X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

def functionXY(Xinput):
  for i, input in enumerate(X):
    if np.array_equal(Xinput, input):
      return y[i]
  return None

print(X[56])
print(y[56])

print(functionXY(X[56]))

shape of X: (98694, 7)
shape of y: (98694,)
[0.05436893 0.54681128 0.41890404 0.90909091 0.0797649  0.2263727
 0.1648951 ]
10.44
10.44


In [16]:
import numpy as np
import random

def objective_function(x):
    # x is a tuple of discrete feature values, corresponding to each feature in X

    # Convert the input tuple to a NumPy array
    x = np.array(x)

    # Find the closest data point in the dataset
    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)

    # Snap the input to the closest data point
    snapped_input = X[closest_index]

    return y[closest_index]

def clip_position(x):
    x = np.array(x)
    
    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)
    
    snapped_input = X[closest_index]
    
    return snapped_input
    
class Particle:
    def __init__(self, num_dimensions, lower_bound, upper_bound, init_position, index):
        self.position = init_position
        self.velocity = np.zeros(num_dimensions)
        self.best_position = np.copy(self.position)
        self.best_fitness = objective_function(self.position)
        self.index = index

class PSO:
    def __init__(self, num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, init_positions):
        self.num_particles = num_particles
        self.num_dimensions = num_dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.num_iterations = num_iterations
        self.particles = [Particle(num_dimensions, lower_bound, upper_bound, init_positions[ind], ind) for ind in range(num_particles)]
        self.global_best_position = np.copy(self.particles[0].position)
        self.global_best_fitness = self.particles[0].best_fitness

    def update_particles(self):
        for particle in self.particles:
            # Update velocities
            particle.velocity = (self.w * particle.velocity
                                 + self.c1 * np.random.rand() * (particle.best_position - particle.position)
                                 + self.c2 * np.random.rand() * (self.global_best_position - particle.position))

            # Update positions
            particle.position = particle.position + particle.velocity

            # Clip positions to stay within bounds
            particle.position = np.clip(particle.position, self.lower_bound, self.upper_bound)
            particle.position = clip_position(particle.position)

    def update_personal_and_global_bests(self):
        for particle in self.particles:
            # Evaluate fitness
            current_fitness = objective_function(particle.position)

            # Update personal best
            if current_fitness > particle.best_fitness:
                particle.best_position = np.copy(particle.position)
                particle.best_fitness = current_fitness

                # Update global best
                if current_fitness > self.global_best_fitness:
                    self.global_best_position = np.copy(particle.position)
                    self.global_best_fitness = current_fitness
            
            print("Particle #", particle.index, ": ")
            print("-> Current Position: ", particle.position)
            print("-> Current Velocity: ", particle.velocity)
            print("-> Current Output: ", functionXY(particle.position))
            print("-> Personal Best Position: ",  particle.best_position)
            
            
        print("---> Global Best Position: ", self.global_best_position, " <---")
        print("---> Global Best Output: ", functionXY(self.global_best_position), " <---")

    def optimize(self):
        for i in range(self.num_iterations):
            print("Iteration #", i, ": ")
            print("x-x-x-x-x-x-x-x-x-x-x")
            self.update_particles()
            self.update_personal_and_global_bests()

        return self.global_best_position, self.global_best_fitness

# Example usage
num_particles = 10
num_dimensions = 7
lower_bound = 0
upper_bound = 1.0
w = 0.8
c1 = 0.8
c2 = 0.8
num_iterations = 10

initial_positions_PSO = random.choices(X, k=num_particles)
print(initial_positions_PSO)


pso = PSO(num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, initial_positions_PSO)
best_position, best_fitness = pso.optimize()

print("Best position found: ", best_position)
print("Best fitness found: ", best_fitness)


[array([0.13009709, 0.29793949, 0.50952936, 0.74747475, 0.02966695,
       0.11360764, 0.09356643]), array([0.26796117, 0.        , 0.        , 0.23232323, 0.00811643,
       0.0581379 , 0.04993007]), array([0.07378641, 0.42225846, 0.42650099, 0.80808081, 0.05429611,
       0.21991293, 0.14307692]), array([0.07961165, 0.39279077, 0.41948476, 0.85858586, 0.05429611,
       0.22763657, 0.16041958]), array([0.17864078, 0.18298256, 0.42287648, 0.62626263, 0.01847187,
       0.10939475, 0.07174825]), array([0.18058252, 0.        , 0.        , 0.56565657, 0.01623286,
       0.06586154, 0.04629371]), array([0.0815534 , 0.60556103, 0.65775891, 0.81818182, 0.05065771,
       0.12442073, 0.09370629]), array([0.11650485, 0.28158769, 0.43345715, 0.75757576, 0.03302547,
       0.14113186, 0.13776224]), array([0.07378641, 0.46377333, 0.46564222, 0.84848485, 0.056815  ,
       0.22763657, 0.11034965]), array([0.21941748, 0.        , 0.        , 0.49494949, 0.01175483,
       0.05420587, 0.04013986])]

-> Current Output:  2.13
-> Personal Best Position:  [0.13980583 0.27722872 0.50929157 0.73737374 0.02686818 0.11023733
 0.09314685]
---> Global Best Position:  [0.12427184 0.40786667 0.66680017 0.7979798  0.03274559 0.104901
 0.13314685]  <---
---> Global Best Output:  3.62  <---
Iteration # 2 : 
x-x-x-x-x-x-x-x-x-x-x
Particle # 0 : 
-> Current Position:  [0.12815534 0.3532041  0.59186437 0.76767677 0.03106633 0.11894397
 0.10629371]
-> Current Velocity:  [-0.00285389  0.05385534  0.07704984  0.02474335  0.00150828 -0.00426554
  0.01939117]
-> Current Output:  3.4
-> Personal Best Position:  [0.12815534 0.3532041  0.59186437 0.76767677 0.03106633 0.11894397
 0.10629371]
Particle # 1 : 
-> Current Position:  [0.10679612 0.5457641  0.77722264 0.93939394 0.04450042 0.11697795
 0.09944056]
-> Current Velocity:  [-0.08025621  0.20024131  0.3000276   0.24491033  0.01305691  0.0282608
  0.02956457]
-> Current Output:  3.81
-> Personal Best Position:  [0.10679612 0.5457641  0.77722264 0.93939

-> Current Output:  3.55
-> Personal Best Position:  [0.10679612 0.5457641  0.77722264 0.93939394 0.04450042 0.11697795
 0.09944056]
Particle # 2 : 
-> Current Position:  [0.08737864 0.48533231 0.57005722 0.82828283 0.04785894 0.17020081
 0.12909091]
-> Current Velocity:  [-0.03547017  0.11385939  0.00701876  0.0506556   0.01373226  0.04672671
  0.02077354]
-> Current Output:  6.0
-> Personal Best Position:  [0.07378641 0.42225846 0.42650099 0.80808081 0.05429611 0.21991293
 0.14307692]
Particle # 3 : 
-> Current Position:  [0.08932039 0.58479282 0.70416919 0.85858586 0.0484187  0.14127229
 0.09090909]
-> Current Velocity:  [-0.00293587  0.0547307   0.10162594  0.0187849   0.00182677 -0.00046452
 -0.0051466 ]
-> Current Output:  6.65
-> Personal Best Position:  [0.07961165 0.39279077 0.41948476 0.85858586 0.05429611 0.22763657
 0.16041958]
Particle # 4 : 
-> Current Position:  [0.09320388 0.66489949 0.83345265 0.81818182 0.04422054 0.08257267
 0.07944056]
-> Current Velocity:  [-0.0209

Particle # 5 : 
-> Current Position:  [0.05242718 0.80621128 0.58117606 0.86868687 0.08172404 0.21232973
 0.20783217]
-> Current Velocity:  [-0.05123015  0.10332186 -0.22197139  0.08668989  0.02874724  0.08238338
  0.06836756]
-> Current Output:  9.78
-> Personal Best Position:  [0.05242718 0.80621128 0.58117606 0.86868687 0.08172404 0.21232973
 0.20783217]
Particle # 6 : 
-> Current Position:  [0.06213592 0.80459795 0.69684007 0.86868687 0.06801008 0.11824182
 0.10083916]
-> Current Velocity:  [-0.0187788  -0.01368945 -0.10509569  0.02734551  0.01133316  0.05328066
  0.04643556]
-> Current Output:  9.15
-> Personal Best Position:  [0.06213592 0.80459795 0.69684007 0.86868687 0.06801008 0.11824182
 0.10083916]
Particle # 7 : 
-> Current Position:  [0.06407767 0.87403795 0.75788857 0.86868687 0.0677302  0.13158264
 0.12965035]
-> Current Velocity:  [-0.0327844   0.18434178 -0.00401416  0.04332856  0.02010296  0.04936566
  0.04266829]
-> Current Output:  8.8
-> Personal Best Position:  [

Particle # 0 : 
-> Current Position:  [0.04660194 0.94838256 0.61804447 0.88888889 0.09235936 0.139868
 0.12937063]
-> Current Velocity:  [-0.02373285  0.08937205 -0.08053253  0.05018984  0.02910742  0.03677646
  0.0324022 ]
-> Current Output:  11.97
-> Personal Best Position:  [0.04660194 0.94838256 0.61804447 0.88888889 0.09235936 0.139868
 0.12937063]
Particle # 1 : 
-> Current Position:  [0.01359223 0.93774359 0.22978608 0.96969697 0.26448363 0.33520573
 0.29090909]
-> Current Velocity:  [-0.02866688  0.03019123 -0.0691314   0.07454834  0.00939354  0.03000536
  0.02850751]
-> Current Output:  24.07
-> Personal Best Position:  [0.01359223 0.93774359 0.22978608 0.96969697 0.26448363 0.33520573
 0.29090909]
Particle # 2 : 
-> Current Position:  [0.04660194 0.94838256 0.61804447 0.88888889 0.09235936 0.139868
 0.12937063]
-> Current Velocity:  [-0.02375453  0.11113926 -0.01679453  0.03417441  0.02441091  0.02965079
  0.01731579]
-> Current Output:  11.97
-> Personal Best Position:  [0.